In [1700]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [1701]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [1702]:
ticker_list = ["aapl", "amzn", "msft"]

ticker = ticker_list[0]

ticker_upper = ticker.upper()

print(ticker_upper)

AAPL


In [1703]:
alphavantage_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}.csv"

In [1704]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek


df["return"] = df["5. adjusted close"].pct_change(1)
df["increase"] = df["return"].apply(lambda x: 1 if x > 0 else 0)

df["log_return"] = np.log(df["return"] + 1)


df.set_index("date", inplace=True)

In [1705]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  return                3063 non-null   float64       
 11  increase              3064 non-null   int64         
 12  log_return            3063 non-null   float64       
dtypes: datet

In [1706]:
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,NaN,0,NaN
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,0.001729,1,0.001727
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,-0.015906,0,-0.016034
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,-0.001849,0,-0.001850
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,0.006648,1,0.006626


In [1707]:
df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,0.001638,1,0.001637
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,-0.011628,0,-0.011696
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,0.020588,1,0.020379
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,-0.001981,0,-0.001983
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,-0.018408,0,-0.018580


In [1708]:
df["return"] = df["return"].shift(-1)
df["increase"] = df["increase"].shift(-1)

In [1709]:
df.head()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,0.001729,1.0,NaN
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,-0.015906,0.0,0.001727
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,-0.001849,0.0,-0.016034
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,0.006648,1.0,-0.001850
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,-0.008822,0.0,0.006626


In [1710]:
df.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return
date,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,-0.011628,0.0,0.001637
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,0.020588,1.0,-0.011696
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,-0.001981,0.0,0.020379
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,-0.018408,0.0,-0.001983
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,NaN,NaN,-0.018580


In [1711]:
df.increase.value_counts()

1.0    1621
0.0    1442
Name: increase, dtype: int64

In [1712]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [1713]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [1714]:
benzinga_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker}_finbert_20100101_20220304.csv"

In [1715]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [1716]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['Stock'])

In [1717]:
df_benzinga["day_of_week"] = df_benzinga["date"].dt.dayofweek

In [1718]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     29077 non-null  object        
 1   Stock        29077 non-null  object        
 2   Positive     29077 non-null  float64       
 3   Negative     29077 non-null  float64       
 4   Neutral      29077 non-null  float64       
 5   date         29077 non-null  datetime64[ns]
 6   day_of_week  29077 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 1.6+ MB


In [1719]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Time to Sell Apple Puts In case you missed the meteoric run in Apple over the last several months, there may still be a way to profit without having to chase the stock at these lofty levels: selling puts. When you sell puts you in a sense become an insurance salesman. You agree to buy the stock at some point in the future, should the stock fall to that level or lower before a given date. For this agreement, you receive a premium up front. With Apple currently trading at $210/share, just off its 52-week high, I propose selling out-of-the-money puts for April or July. The April $190 contract bids at $8.15 and the July $180 contract bids at $10.25. I chose such long-dated contracts for their healty premiums. Just a reminder that the first rule of thumb with selling puts is to only sell puts on a stock you would be willing to own. Since I firmly believe in Apple's fundamentals and future business prospects, I would willingly buy the stock at $190 in April. However, nothing is certain, and Apple shares currently have a lot of positive news already ""baked in"". If the upcoming tablet is a flop or never arrives at all, or if the company somehow misses earnings forecasts, the stock price may fall sharply. Since I agree with most analysts that Apple will earn roughly $11-12 in non-GAAP EPS in FY2010 and that a $240-260 price target is reasonable, selling out-of-the-money puts appears to be a great way to generate income and set the price you would be willing to pay for Apple shares. Therefore, it is my bet that Apple will stay above $190 by April and $180 by July and the put options will expire worthless, giving the seller a nice premium for either contract.",2010-01-02,0.085465,0.15849,0.756044,2010-01-02,5


In [1720]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     29077 non-null  object        
 1   Stock        29077 non-null  object        
 2   Positive     29077 non-null  float64       
 3   Negative     29077 non-null  float64       
 4   Neutral      29077 non-null  float64       
 5   date         29077 non-null  datetime64[ns]
 6   day_of_week  29077 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 1.6+ MB


In [1721]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_benzinga.day_of_week.value_counts()

1    6598
2    5977
0    5639
3    5184
4    4465
6    667 
5    547 
Name: day_of_week, dtype: int64

In [1722]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Time to Sell Apple Puts In case you missed the meteoric run in Apple over the last several months, there may still be a way to profit without having to chase the stock at these lofty levels: selling puts. When you sell puts you in a sense become an insurance salesman. You agree to buy the stock at some point in the future, should the stock fall to that level or lower before a given date. For this agreement, you receive a premium up front. With Apple currently trading at $210/share, just off its 52-week high, I propose selling out-of-the-money puts for April or July. The April $190 contract bids at $8.15 and the July $180 contract bids at $10.25. I chose such long-dated contracts for their healty premiums. Just a reminder that the first rule of thumb with selling puts is to only sell puts on a stock you would be willing to own. Since I firmly believe in Apple's fundamentals and future business prospects, I would willingly buy the stock at $190 in April. However, nothing is certain, and Apple shares currently have a lot of positive news already ""baked in"". If the upcoming tablet is a flop or never arrives at all, or if the company somehow misses earnings forecasts, the stock price may fall sharply. Since I agree with most analysts that Apple will earn roughly $11-12 in non-GAAP EPS in FY2010 and that a $240-260 price target is reasonable, selling out-of-the-money puts appears to be a great way to generate income and set the price you would be willing to pay for Apple shares. Therefore, it is my bet that Apple will stay above $190 by April and $180 by July and the put options will expire worthless, giving the seller a nice premium for either contract.",2010-01-02,0.085465,0.15849,0.756044,2010-01-02,5


In [1723]:
df_benzinga["in_index"] = df_benzinga["Stock"].apply(lambda x: True if x in df.index else False)

In [1724]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    667
5    547
0    109
4    22 
3    13 
1    10 
2    4  
Name: day_of_week, dtype: int64

In [1725]:
df.index.get_loc('2010-01-04')


0

In [1726]:
from datetime import date, datetime, timedelta

In [1727]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [1728]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Monday
                days = timedelta(1)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Monday
                days = timedelta(2)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Monday
                days = timedelta(3)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Tuesday
                days = timedelta(1)
                idx_lookup = (dt_time + days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(e3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc

In [1729]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_benzinga["backfill_date"] = df_benzinga.apply(update_date, axis=1)


2010-01-04
2010-01-11
2010-01-11
2010-01-18
'2010-01-18'
2010-01-19
2010-01-19
2010-01-19
2010-01-19
2010-02-01
2010-02-01
2010-02-15
'2010-02-15'
2010-02-16
2010-02-16
2010-02-16
2010-02-16
2010-02-16
2010-02-16
2010-02-16
2010-02-16
2010-02-22
2010-03-01
2010-03-08
2010-03-08
2010-03-08
2010-03-15
2010-03-29
2010-03-29
2010-02-01
2010-04-05
2010-04-05
2010-04-19
2010-05-03
2010-05-10
2010-05-10
2010-06-01
2010-06-01
2010-06-07
2010-06-28
2010-06-28
2010-07-05
'2010-07-05'
2010-07-12
2010-07-19
2010-07-19
2010-07-19
2010-07-26
2010-07-26
2010-07-26
2010-07-26
2010-08-09
2010-08-16
2010-08-30
2010-08-30
2010-09-06
'2010-09-06'
2010-09-07
2010-09-07
2010-10-04
2010-10-18
2010-10-18
2010-10-11
2010-11-08
2010-11-22
2010-11-22
2010-11-22
2010-12-06
2010-12-06
2010-12-13
2010-12-20
2010-12-20
2010-12-20
2010-12-27
2011-01-10
2011-01-10
2011-01-17
'2011-01-17'
2011-01-17
'2011-01-17'
2011-01-18
2011-01-18
2011-01-18
2011-01-18
2011-01-18
2011-01-18
2011-01-18
2011-01-24
2011-01-24
2011-01-2

In [1730]:
ERROR_COUNTER

95

In [1731]:
df_benzinga["in_index"] = df_benzinga["backfill_date"].apply(lambda x: True if x in df.index else False)

In [1732]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    43
5    37
0    15
3    13
1    10
2    4 
Name: day_of_week, dtype: int64

In [1733]:
df_benzinga[df_benzinga.in_index==False].shape

(122, 9)

In [1734]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [1735]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [1736]:
df_benzinga = df_benzinga[["backfill_date", "Positive", "Negative", "Neutral"]]

In [1737]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  29077 non-null  object 
 1   Positive       29077 non-null  float64
 2   Negative       29077 non-null  float64
 3   Neutral        29077 non-null  float64
dtypes: float64(3), object(1)
memory usage: 908.8+ KB


In [1738]:
df_benzinga.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-04,0.085465,0.158490,0.756044
1,2010-01-04,0.044536,0.688256,0.267208
2,2010-01-04,0.943631,0.022050,0.034319
3,2010-01-04,0.016127,0.955271,0.028602
4,2010-01-04,0.818431,0.036988,0.144581


In [1739]:
df_benzinga.rename(columns={"backfill_date": "date"}, inplace=True)

In [1740]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [1741]:
# df_benzinga = clean_tweet(df_benzinga)

In [1742]:
df_benzinga.set_index("date", inplace=True)

In [1743]:
df_benzinga.head()

,Positive,Negative,Neutral
date,,,
2010-01-04,0.085465,0.158490,0.756044
2010-01-04,0.044536,0.688256,0.267208
2010-01-04,0.943631,0.022050,0.034319
2010-01-04,0.016127,0.955271,0.028602
2010-01-04,0.818431,0.036988,0.144581


In [1744]:
df_benzinga_duplicated_index = df_benzinga[df_benzinga.index.duplicated(keep=False)]

In [1745]:
df_benzinga_duplicated_index.shape[0]

28987

In [1746]:
df_benzinga_nonduplicated_index = df_benzinga[~df_benzinga.index.duplicated(keep=False)]

In [1747]:
df_benzinga_nonduplicated_index.shape[0]

90

In [1748]:
# df_twint.groupby('date')['negative'].mean()

df_benzinga_groupby = df_benzinga.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [1749]:
df_benzinga_groupby

,negative,nuetral,positive
date,,,
2010-01-04,0.210025,0.405298,0.384677
2010-01-05,0.243738,0.529363,0.226900
2010-01-06,0.077748,0.422044,0.500207
2010-01-07,0.251646,0.076096,0.672258
2010-01-08,0.227638,0.496481,0.275881
...,...,...,...
2022-02-28,0.442030,0.409206,0.148764
2022-03-01,0.468003,0.407704,0.124294
2022-03-02,0.442909,0.424314,0.132777


In [1750]:
df_merge = pd.merge(df, df_benzinga_groupby, how="left", left_index=True, right_index=True)

In [1751]:
df_merge.shape

(3064, 16)

In [1752]:
df_merge.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [1753]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2012-08-10,618.71,621.76,618.7000,621.70,19.091146,6962100.0,0.00,1.0,2012-08-10,4,0.013350,1.0,0.001561,NaN,NaN,NaN
2014-07-31,97.16,97.45,95.3300,95.60,21.409463,56843000.0,0.00,1.0,2014-07-31,3,0.005544,1.0,-0.026324,NaN,NaN,NaN
2016-06-22,96.25,96.89,95.3500,95.55,22.217966,29219122.0,0.00,1.0,2016-06-22,2,0.005756,1.0,-0.003761,NaN,NaN,NaN
2016-11-04,108.53,110.25,108.1100,108.84,25.576562,30836997.0,0.00,1.0,2016-11-04,4,0.014425,1.0,-0.009055,NaN,NaN,NaN
2016-12-23,115.59,116.52,115.5900,116.52,27.381303,14249484.0,0.00,1.0,2016-12-23,4,0.006351,1.0,0.001976,NaN,NaN,NaN
2017-11-24,175.10,175.50,174.6459,174.97,41.780905,14026519.0,0.00,1.0,2017-11-24,4,-0.005029,0.0,0.000057,NaN,NaN,NaN
2018-02-23,173.67,175.65,173.5400,175.50,42.076261,33329232.0,0.00,1.0,2018-02-23,4,0.019772,1.0,0.017242,NaN,NaN,NaN
2018-03-02,172.80,176.30,172.4500,176.21,42.246484,38453950.0,0.00,1.0,2018-03-02,4,0.003462,1.0,0.006890,NaN,NaN,NaN
2018-03-09,177.96,180.00,177.3900,179.98,43.150344,31385134.0,0.00,1.0,2018-03-09,4,0.009668,1.0,0.017035,NaN,NaN,NaN


In [1754]:
df.shape

(3064, 13)

In [1755]:
df_benzinga_groupby.shape

(3112, 3)

In [1756]:
# df_merge.fillna(0.0, inplace=True)
df_merge.fillna(method="ffill", limit=1, inplace=True)

In [1757]:
df_merge[df_merge.negative.isnull()].shape

(0, 16)

In [1758]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  return                3064 non-null   float64       
 11  increase              3064 non-null   float64       
 12  log_return            3063 non-null   float64       
 13  negative

In [1759]:
df_merge.fillna(0.0, inplace=True)

In [1760]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,0.001729,1.0,0.000000,0.210025,0.405298,0.384677
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,-0.015906,0.0,0.001727,0.243738,0.529363,0.226900
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,-0.001849,0.0,-0.016034,0.077748,0.422044,0.500207
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,0.006648,1.0,-0.001850,0.251646,0.076096,0.672258
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,-0.008822,0.0,0.006626,0.227638,0.496481,0.275881


In [1761]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [1762]:
df_merge.shape

(3064, 16)

In [1763]:
# df_merge[(df_merge["negative"]==0.0) & (df_merge["nuetral"]==0.0) & (df_merge["positive"]==0.0)].shape

In [1764]:
# df_merge = df_merge[~(df_merge["negative"]==0.0) & ~(df_merge["nuetral"]==0.0) & ~(df_merge["positive"]==0.0)].copy()

In [1765]:
X = df_merge[["log_return", "6. volume", "positive"]]
y = df_merge["increase"]

In [1766]:
y.head()

date
2010-01-04    1.0
2010-01-05    0.0
2010-01-06    0.0
2010-01-07    1.0
2010-01-08    0.0
Name: increase, dtype: float64

In [1767]:
SPLIT = int(0.9 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [1768]:
X_train

,log_return,6. volume,positive
date,,,
2010-01-04,0.000000,17633200.0,0.384677
2010-01-05,0.001727,21496600.0,0.226900
2010-01-06,-0.016034,19720000.0,0.500207
2010-01-07,-0.001850,17040400.0,0.672258
2010-01-08,0.006626,15986100.0,0.275881
...,...,...,...
2020-12-08,0.005078,82225512.0,0.181166
2020-12-09,-0.021125,115089193.0,0.204811
2020-12-10,0.011918,81312170.0,0.026226


In [1769]:
from sklearn.ensemble import RandomForestClassifier

In [1770]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [1771]:
preds = rfc.predict(X_test)

In [1772]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [1773]:
matrix = confusion_matrix(y_test, preds)

In [1774]:
matrix

array([[69, 82],
       [64, 92]])

In [1775]:
score = accuracy_score(y_test, preds)

In [1776]:
score

0.5244299674267101

In [1777]:
y_test.value_counts()

1.0    156
0.0    151
Name: increase, dtype: int64

In [1778]:
y_test.value_counts(normalize=True)

1.0    0.508143
0.0    0.491857
Name: increase, dtype: float64

In [1779]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

         0.0       0.52      0.46      0.49       151
         1.0       0.53      0.59      0.56       156

    accuracy                           0.52       307
   macro avg       0.52      0.52      0.52       307
weighted avg       0.52      0.52      0.52       307



In [1780]:
corr_matrix = df_merge[["log_return", "positive", "increase"]].corr()
print (corr_matrix)

            log_return  positive  increase
log_return  1.000000    0.260377 -0.029765
positive    0.260377    1.000000  0.009690
increase   -0.029765    0.009690  1.000000


In [1781]:
from statsmodels.tsa.stattools import grangercausalitytests

In [1782]:
# #perform Granger-Causality test
# grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[1])

In [1783]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,0.001729,1.0,0.000000,0.210025,0.405298,0.384677
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,-0.015906,0.0,0.001727,0.243738,0.529363,0.226900
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,-0.001849,0.0,-0.016034,0.077748,0.422044,0.500207
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,0.006648,1.0,-0.001850,0.251646,0.076096,0.672258
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,-0.008822,0.0,0.006626,0.227638,0.496481,0.275881


In [1784]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,-0.011628,0.0,0.001637,0.442030,0.409206,0.148764
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,0.020588,1.0,-0.011696,0.468003,0.407704,0.124294
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,-0.001981,0.0,0.020379,0.442909,0.424314,0.132777
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,-0.018408,0.0,-0.001983,0.317731,0.523530,0.158739
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,-0.018408,0.0,-0.018580,0.255765,0.641111,0.103125


In [1785]:
df_merge["return"].mean(), df_merge["return"].std()

(0.0012005543577513866, 0.017678905701010397)

In [1786]:
df_merge["log_return"].mean(), df_merge["log_return"].std()

(0.0010496941616316064, 0.01768794571922299)

In [1787]:
df_merge["increase"] = df_merge["increase"].shift(1)
df_merge["return"] = df_merge["return"].shift(1)

In [1788]:
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,NaN,NaN,0.000000,0.210025,0.405298,0.384677
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,0.001729,1.0,0.001727,0.243738,0.529363,0.226900
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,-0.015906,0.0,-0.016034,0.077748,0.422044,0.500207
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,-0.001849,0.0,-0.001850,0.251646,0.076096,0.672258
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,0.006648,1.0,0.006626,0.227638,0.496481,0.275881


In [1789]:
df_merge.tail()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,return,increase,log_return,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,0.001638,1.0,0.001637,0.442030,0.409206,0.148764
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,-0.011628,0.0,-0.011696,0.468003,0.407704,0.124294
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,0.020588,1.0,0.020379,0.442909,0.424314,0.132777
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,-0.001981,0.0,-0.001983,0.317731,0.523530,0.158739
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,-0.018408,0.0,-0.018580,0.255765,0.641111,0.103125


In [1790]:
df_merge.fillna(0.0, inplace=True)

In [1791]:
# df_merge.to_csv(f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}_WITH_BENZINGA_FINBERT_SA.csv")